In [346]:
import requests as re
import pandas as pd
import datetime
import altair as alt
import altair_latimes as lat

In [347]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Download
Source: [California Water Watch](https://cww.water.ca.gov/), PRISM

Some parameters

In [348]:
today = datetime.date.today()
today

datetime.date(2024, 2, 8)

In [349]:
one_year_ago = datetime.datetime.today() - datetime.timedelta(days=365)
one_year_ago

datetime.datetime(2023, 2, 8, 12, 28, 50, 705787)

In [350]:
atlasID="Statewide"
varKey="PPT_YTD"

Query

In [351]:
url = f'https://cww.water.ca.gov/service/prism/huc8/precipstats?date={today}&atlasID={atlasID}&varKey={varKey}'

In [352]:
data = re.get(url).json()['data']

Query one year ago

In [353]:
last_year_url = f'https://cww.water.ca.gov/service/prism/huc8/precipstats?date={one_year_ago}&atlasID={atlasID}&varKey={varKey}'

In [354]:
last_year_data = re.get(last_year_url).json()['data']

### Parse

In [355]:
def parsePrecipStats(data):
    columns = data['seriesNames']
    columns.insert(0, "date")
    
    df = pd.DataFrame(data['series'], columns=columns)
    df.columns = df.columns.str.lower()
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [356]:
df = parsePrecipStats(data)

In [357]:
last_year_df = parsePrecipStats(last_year_data)

## Get toplines

In [358]:
prcp_min=data['min']
prcp_min

3.15

In [359]:
prcp_avg=data['avg']
prcp_avg

13.26

In [360]:
prcp_max=data['max']
prcp_max

25.39

In [361]:
year_end_avg = data['yearEndAvg']
year_end_avg

23.75

### Plot

In [362]:
gauge = df[
    (~df.current.isna())
]

In [363]:
gauge = gauge[gauge.date == gauge.date.max()]

In [364]:
gauge["record"] = prcp_max

In [365]:
gauge["pct_of_avg"] = (gauge["current"] / gauge["average"]) * 100

Last year

In [366]:
total_last_year = last_year_df[ last_year_df.date == gauge.date.max() ].iloc[0]["current"]

In [367]:
gauge["one_year_ago"] = total_last_year

In [368]:
gauge

,date,current,average,record,pct_of_avg,one_year_ago
129,2024-02-07,13.37,13.1,25.39,102.061069,18.8


In [369]:
alt.Chart(
    pd.melt(gauge, id_vars="date", value_vars=["current","average","one_year_ago"])
).mark_bar().encode(
    x="value",
    y=alt.Y("variable", sort="-x"),
    color="variable"
).properties(
    title="Year-to-date precipitation in California compared to average",
    width=600,
    height=200
)

alt.Chart(...)

In [370]:
line_chart = pd.melt(df, id_vars="date", value_vars=["current","average"])

In [371]:
alt.Chart(line_chart).mark_line().encode(
    x="date:T",
    y="value",
    color="variable",
).properties(
    title=f"California precipitation in {today.year} compared to average",
    width=600
)

alt.Chart(...)

### Export

In [372]:
gauge.to_csv("../../data/processed/precipitation/latest.csv", index=False)